In [1]:
import pandas as pd

from pprint import pprint
from contextlib import contextmanager

from io import StringIO
import re


@contextmanager
def display_custom_rows(num_rows):
    original_max_rows = pd.get_option('display.max_rows')
    pd.set_option('display.max_rows', num_rows)
    try:
        yield
    finally:
        pd.set_option('display.max_rows', original_max_rows)


## Clean Data

In [2]:
with open("pairs/README", "r") as f:
    raw_readme = f.read().split("\n")[29:]

cols = raw_readme.pop(0)
cols = ["pair"] + [c.strip() for c in cols.replace("\t","|").split("|") if c] + ["group"]

group_num = 0
rows = []
for line in raw_readme:
    if line.strip() == '':
        group_num += 1
        continue
    rows.append([c.strip() for c in line.replace(" " * 5, "\t").replace("\t","|").split("|") if c] + [group_num])

pairs_info_df = pd.DataFrame(rows, columns = cols)

# with display_custom_rows(110):
#     display(readme_df)

In [3]:
pair_np_arrs = dict()
for pair in pairs_info_df.pair:
    file = f"pairs/{pair}.txt"
    with open(file, "r") as f:
        raw_str = re.sub(r'[^\S\n]+', ' ', f.read())
        buffer = StringIO(raw_str)    
    pair_data = pd.read_csv(buffer, sep=None, engine="python", header=None).dropna(axis=1).to_numpy()
    pair_np_arrs[pair] = pair_data


In [4]:
pairs_info_df = pairs_info_df.assign(
    sample_size = lambda x: x.pair.map(lambda y: pair_np_arrs[y].shape[0]),
    num_features = lambda x: x.pair.map(lambda y: pair_np_arrs[y].shape[1]),
    weight = 1.0,
    pair_num = lambda x: x.pair.str[5:].astype(int),
)

In [5]:
pairs_info_df

,pair,var 1,var 2,dataset,ground truth,group,sample_size,num_features,weight,pair_num
0,pair0001,Altitude,Temperature,DWD,->,1,349,2,1.0,1
1,pair0002,Altitude,Precipitation,DWD,->,1,349,2,1.0,2
2,pair0003,Longitude,Temperature,DWD,->,1,349,2,1.0,3
3,pair0004,Altitude,Sunshine hours,DWD,->,1,349,2,1.0,4
4,pair0005,Age,Length,Abalone,->,2,4177,2,1.0,5
...,...,...,...,...,...,...,...,...,...,...
103,pair0104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,48,109,2,1.0,104
104,pair0105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,49,1000,10,1.0,105
105,pair0106,time required for one round,voltage,D. Janzing,<-,50,114,2,1.0,106
106,pair0107,strength of contrast,answer correct or not,"Schuett, edited by D. Janzing",->,51,240,2,1.0,107


In [6]:
similar_pairs = [
    [49,50,51],
    [56, 57, 58, 59, 60, 61, 62, 63],
    [81, 82, 83],
    [89, 90],
    [97, 98],   
]

In [7]:
for spairs in similar_pairs:
    weight = 1/len(spairs)
    pairs_info_df.loc[pairs_info_df.pair_num.isin(spairs), "weight"] = weight

## Test with RESIT and GZIP RESIT

In [8]:
import sys
sys.path.append("..")

from resit import MaxNormNCDRESIT, LevenshteinRESIT
from lingam import RESIT

from tqdm import tqdm
import numpy as np
from joblib import Parallel, delayed


from sklearn.ensemble import RandomForestRegressor

In [9]:
pairs = pairs_info_df.loc[pairs_info_df.num_features == 2, "pair"]

In [10]:
tuple([0,1]) == (0,1)

True

In [11]:
tuple(np.random.choice(2, size=2, replace=False))

(0, 1)

In [18]:
import sys

sys.path.append("..")
from resit import MaxNormNCDRESIT
import gzip, bz2, lzma, zstandard
import time
import itertools


def order2arrow(order):
    return "->" if tuple(order) == (0, 1) else "<-"


def process_pair(pair, sample_size):
    arr = pair_np_arrs[pair]
    # if sample_size > 0:
    #     sample_idx = np.random.choice(
    #         arr.shape[0], min(arr.shape[0], sample_size), replace=False
    #     )
    #     arr = arr[sample_idx]
    arr_norm = (arr - arr.mean(axis=0)) / arr.std(axis=0)

    seed = 2024

    model = lambda: RandomForestRegressor(random_state=69)
    model_configs = {
        "resit": lambda: RESIT(model(), random_state=seed),
        "resit_gzip": lambda: MaxNormNCDRESIT(
            model(), compressor=gzip, random_state=seed, mi_agg=np.mean
        ),
        "resit_bz2": lambda: MaxNormNCDRESIT(
            model(), compressor=bz2, random_state=seed, mi_agg=np.mean
        ),
        "resit_lzma": lambda: MaxNormNCDRESIT(
            model(), compressor=lzma, random_state=seed, mi_agg=np.mean
        ),
        "resit_zstandard": lambda: MaxNormNCDRESIT(
            model(), compressor=zstandard, random_state=seed, mi_agg=np.mean
        ),
    }

    ground_truth = pairs_info_df.loc[pairs_info_df.pair == pair, "ground truth"].item()
    random = np.random.choice(2, size=2, replace=False)

    result = {
        "pair": pair,
        "sample_size": sample_size,
        "random_acc": order2arrow(random) == ground_truth,
    }
    for resit_model_name, resit_model in model_configs.items():
        start_time = time.time()
        causal_order = resit_model().fit(arr).causal_order_
        end_time = time.time()
        total_time = end_time - start_time

        result[f"{resit_model_name}_is_correct"] = order2arrow(causal_order) == ground_truth
        result[f"time_{resit_model_name}"] = total_time

        start_time = time.time()
        causal_order = resit_model().fit(arr_norm).causal_order_
        end_time = time.time()
        total_time = end_time - start_time

        result[f"{resit_model_name}_norm_is_correct"] = (
            order2arrow(causal_order) == ground_truth
        )
        result[f"time_{resit_model_name}_norm"] = total_time

        return result


# Assuming you have defined 'pairs' elsewhere

sample_sizes = [-1]
param_combinations = list(itertools.product(pairs, sample_sizes))
results = Parallel(n_jobs=30)(
    delayed(process_pair)(pair, sample_size)
    for pair, sample_size in tqdm(param_combinations)
)

 59%|█████▉    | 60/102 [00:03<00:02, 16.22it/s]/home/hong/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hong/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/hong/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hong/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 102/102 [00:05<00:00, 17.14it/s]


KeyboardInterrupt: 

In [17]:
pd.DataFrame(results).set_index("pair").mean()

resit_acc        0.509804
compresit_acc    0.549020
random_acc       0.450980
dtype: float64